In [119]:
import pandas as pd
import numpy as np
import random

In [179]:
class Card:
    SUIT_MAP = {
        'D': 'Diamonds',
        'S': 'Spades',
        'C': 'Clubs',
        'H': 'Hearts'
    }

    RANK_MAP = {
        '2': "Two",
        '3': "Three",
        '4': "Four",
        '5': "Five",
        '6': "Six",
        '7': "Seven",
        '8': "Eight",
        '9': "Nine",
        '10': "Ten",
        'J': "Jack",
        'Q': "Queen",
        'K': "King",
        'A': "Ace"
    }

    COLOUR_MAP = {
        'R': "Red",
        'B': "Black"
    }

    def __init__(self, suit: str, color: str, rank: str):
        self.suit = suit
        self.color = color
        self.rank = rank

    def getSuit(self) -> str:
        return self.suit

    def getColor(self) -> str:
        return self.color

    def getRank(self) -> str:
        return self.rank

    def printCard(self):
        name = Card.RANK_MAP[self.rank] + " of " + Card.SUIT_MAP[self.suit]
        print(name)

    def getCard(self):
        return (self.rank + " of " + Card.SUIT_MAP[self.suit])

In [180]:
class Deck:

    SUITS = ['D', 'H', 'C', 'S']
    
    RANKS = {
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
        '10': 10,
        'J': 11,
        'Q': 12,
        'K': 13,
        'A': 14
    }

    COLORS = ['R','B']

    COLOUR_SUIT_MAP = {
        'R':['H', 'D'],
        'B': ['S', 'C']
    }

    def __init__(self):
        deck = list()
        for color in Deck.COLORS:
            for rank in Deck.RANKS:
                for suit in Deck.COLOUR_SUIT_MAP[color]:
                    deck.append(Card(suit,color,rank))
        random.shuffle(deck)
        self.deck = deck
        self.size = len(deck)

    def getDeck(self):
        return self.deck

    def drawCards(self, numberOfDraws: int = 1) -> list:
        assert self.size >= numberOfDraws, "Deck doesn't have enough cards!"

        np.random.seed()
        cardsDrawn = []
        for i in range(numberOfDraws):
            if self.size!=1:
                location = np.random.randint((self.size)-1)
            else:
                location = 0
            cardsDrawn.append(self.deck.pop(location))
            self.size = self.size - 1
        return cardsDrawn

    def shuffleDeck(self):
        random.shuffle(self.deck)


In [181]:
class Hand:

    def __init__(self, deck: Deck, numberOfCards: int = 3):
        self.hand = deck.drawCards(numberOfCards)

    def getHand(self):
        return self.hand
    

In [182]:
# TODO demonstration

deck = Deck()

print("Drawing 20 cards")
drawn = []
left = []
for item in deck.drawCards(20):
    drawn.append(item.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards drawn:", drawn)
print("Deck remaining:", left)
print()

print("Drawing a hand of 3 cards")
hand = Hand(deck)
handCards = []
left = []
for card in hand.getHand():
    handCards.append(card.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards in hand:", handCards)
print("Deck remaining:", left)
print()

Drawing 20 cards
Deck size remaining 32
Cards drawn: ['Q of Hearts', '7 of Spades', 'J of Clubs', 'A of Diamonds', 'J of Hearts', '3 of Spades', 'A of Hearts', '5 of Hearts', 'A of Spades', 'K of Spades', '8 of Hearts', '9 of Hearts', '7 of Diamonds', 'K of Hearts', '10 of Clubs', 'K of Diamonds', '7 of Hearts', '6 of Hearts', '10 of Diamonds', 'Q of Diamonds']
Deck remaining: ['6 of Clubs', '3 of Clubs', '3 of Diamonds', '2 of Hearts', '10 of Spades', '8 of Diamonds', '5 of Clubs', '4 of Diamonds', '3 of Hearts', '5 of Diamonds', 'A of Clubs', 'Q of Spades', '6 of Diamonds', 'K of Clubs', '6 of Spades', '2 of Diamonds', '7 of Clubs', '2 of Spades', '4 of Spades', '4 of Clubs', '5 of Spades', 'Q of Clubs', '8 of Clubs', 'J of Spades', '10 of Hearts', '2 of Clubs', '9 of Clubs', 'J of Diamonds', '8 of Spades', '9 of Spades', '4 of Hearts', '9 of Diamonds']

Drawing a hand of 3 cards
Deck size remaining 29
Cards in hand: ['8 of Diamonds', '6 of Spades', '6 of Diamonds']
Deck remaining: [

In [ ]:
class Game:

    HAND_RANKINGS = {}

    def __init__(self, noOfPlayers: int, jokers: list = None):
        self.noOfPlayers = noOfPlayers
        self.hands = self.generateHands()
        self.jokers = jokers

    def generateHands(self):
        pass

    